In [ ]:
# ---
# title: 07. The Efficient Frontier
# tags: [Optimization, Scipy, Quant]
# difficulty: Advanced
# ---

import pandas as pd
import numpy as np
import scipy.optimize as sco
import matplotlib.pyplot as plt
from pathlib import Path

# Setup
processed_path = Path("../data/silver")
returns_file = max(list(processed_path.glob("market_returns_*.parquet")), key=lambda f: f.stat().st_mtime)
df_returns = pd.read_parquet(returns_file)

# Subset for visualization speed
subset_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'JPM', 'JNJ', 'XOM', 'WMT']
df = df_returns[subset_tickers]
mean_returns = df.mean() * 252
cov_matrix = df.cov() * 252
num_assets = len(subset_tickers)

# Calculating the Efficient Frontier

The **Efficient Frontier** is the set of portfolios that offer the highest expected return for a defined level of risk.

We use `scipy.optimize` to find these points mathematically.

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, rf_rate):
    p_ret, p_std = portfolio_performance(weights, mean_returns, cov_matrix)
    return - (p_ret - rf_rate) / p_std

# Optimization Config
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0.0, 1.0) for asset in range(num_assets))
init_guess = num_assets * [1. / num_assets,]
rf_rate = 0.04

# Solve for Max Sharpe
opt_result = sco.minimize(neg_sharpe_ratio, init_guess, args=(mean_returns, cov_matrix, rf_rate),
                          method='SLSQP', bounds=bounds, constraints=constraints)

opt_weights = opt_result.x
opt_ret, opt_std = portfolio_performance(opt_weights, mean_returns, cov_matrix)

print(f"Max Sharpe: {-(opt_result.fun):.2f}")
print(f"Return: {opt_ret:.2%}, Volatility: {opt_std:.2%}")

In [ ]:
# Monte Carlo Simulation to Visualize
results = np.zeros((3, 5000))
for i in range(5000):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    p_ret, p_std = portfolio_performance(weights, mean_returns, cov_matrix)
    results[0,i] = p_std
    results[1,i] = p_ret
    results[2,i] = (p_ret - rf_rate) / p_std

plt.figure(figsize=(10, 6))
plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='viridis', s=10)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(opt_std, opt_ret, c='red', s=200, marker='*', label='Optimal Portfolio')
plt.title('Efficient Frontier Simulation')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.legend()
plt.grid()
plt.show()